### Predicting the Severity of Automobile Accidents in Seattle, Washington ###

In this first week, you will discover your
project objectives, find your dataset that you will use for this capstone project, and publish your
dataset on GitHub.

In the second week, you will build your machine
learning solution.

In the third week,
you will finalize your model and be ready
to submit your work.

To complete capstone,
you will be working on a case study which is to predict the severity
of an accident.
Now, wouldn't it be great if there were something in place that could warn you, 
given the weather and the road conditions,
about the possibility of you getting into a car accident and how severe it would be,
so that you would drive more carefully or even change your travel plans?
Let's use our shared data for Seattle, Washington as an example of how to deal with the accidents data.

In [1]:
# Import common packages for Data Science applications.
import io
import itertools
import matplotlib as mpl
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import pylab as pl
import scipy
import scipy.optimize as opt
import seaborn as sns
import sklearn
import sklearn.linear_model
import sys
from matplotlib.ticker import NullFormatter
from scipy import optimize
from scipy.optimize import curve_fit
from sklearn import linear_model
from sklearn import metrics
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [2]:
# NOTE: >>> help(pd.options.display. <TAB>
# pd.options.display.chop_threshold      pd.options.display.float_format        pd.options.display.max_info_columns    pd.options.display.notebook_repr_html
# pd.options.display.colheader_justify   pd.options.display.html                pd.options.display.max_info_rows       pd.options.display.pprint_nest_depth
# pd.options.display.column_space        pd.options.display.large_repr          pd.options.display.max_rows            pd.options.display.precision
# pd.options.display.date_dayfirst       pd.options.display.latex               pd.options.display.max_seq_items       pd.options.display.show_dimensions
# pd.options.display.date_yearfirst      pd.options.display.max_categories      pd.options.display.memory_usage        pd.options.display.unicode
# pd.options.display.encoding            pd.options.display.max_columns         pd.options.display.min_rows            pd.options.display.width
# pd.options.display.expand_frame_repr   pd.options.display.max_colwidth        pd.options.display.multi_sparse        

# Create a list of display options.
list_of_display_options_fully_qualified_names = str(\
"pd.options.display.chop_threshold, pd.options.display.float_format, pd.options.display.max_info_columns, pd.options.display.notebook_repr_html, \
pd.options.display.colheader_justify, pd.options.display.html, pd.options.display.max_info_rows, pd.options.display.pprint_nest_depth, \
pd.options.display.column_space, pd.options.display.large_repr, pd.options.display.max_rows, pd.options.display.precision, \
pd.options.display.date_dayfirst, pd.options.display.latex, pd.options.display.max_seq_items, pd.options.display.show_dimensions, \
pd.options.display.date_yearfirst, pd.options.display.max_categories, pd.options.display.memory_usage, pd.options.display.unicode, \
pd.options.display.encoding, pd.options.display.max_columns, pd.options.display.min_rows, pd.options.display.width, \
pd.options.display.expand_frame_repr, pd.options.display.max_colwidth, pd.options.display.multi_sparse").split(sep=', ')

# Initialize an empty list to store all the short names for display options.
list_of_display_options_short_names = list()
# For each fully qualified option name,
# get the option's short name and add it to the list of short names.
for fully_qualified_option_name in list_of_display_options_fully_qualified_names:
    # Get short option name.
    short_option_name = fully_qualified_option_name.split(sep='.')[-1]
    
    # Add short option name to list of display option short names.
    list_of_display_options_short_names.append(short_option_name)

# Define dictionary of display option settings.
dict_of_display_option_settings_short_names=\
{"max_info_columns": 500,\
"colheader_justify": "right",\
"max_info_rows": 1000,\
"column_space": 500,\
"max_rows": 1000,\
"precision": 9,\
"max_seq_items": 1000000000,\
"show_dimensions": True,\
"max_categories": 100,\
"memory_usage": True,\
"max_columns": 500,\
"max_colwidth": 500,\
"float_format": lambda x: '%.9f' % x}

# Set pandas display options using dictionary of short names,
# and display the options/value pairs.
print("Setting display options...")
for key in list(dict_of_display_option_settings_short_names.keys()):
    # Set display option.
    pd.set_option(key, dict_of_display_option_settings_short_names[key])
    # Print display option name and value.
    print(key, ": ", pd.get_option(key), sep='')

Setting display options...
max_info_columns: 500
colheader_justify: right
max_info_rows: 1000
column_space: 500
max_rows: 1000
precision: 9
max_seq_items: 1000000000
show_dimensions: True
max_categories: 100
memory_usage: True
max_columns: 500
max_colwidth: 500
float_format: <function <lambda> at 0x7f9924faf700>


In [3]:
# Attribute Information URL: https://www.seattle.gov/Documents/Departments/SDOT/GIS/Collisions_OD.pdf
# Read the Collisions Data CSV file and store it as a DataFrame.
# url="https://opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv" # HTTPError at 202009151050, using local copy of .csv instead.
# print(os.listdir("..")) # Print list of contents of current working directory.
local_path_to_csv = "../Collisions.csv"
df=pd.read_csv(local_path_to_csv, low_memory=False)

In [4]:
# View the first few rows of the collisions DataFrame.
df.head()

X            Y  OBJECTID  INCKEY  COLDETKEY   REPORTNO  \
0 -122.320757054 47.609407946         1  328476     329976    EA08706   
1 -122.319560827 47.662220664         2  328142     329642    EA06882   
2 -122.327524508 47.604393273         3   20700      20700    1181833   
3 -122.327524934 47.708621579         4  332126     333626  M16001640   
4 -122.292120049 47.559009080         5  328238     329738    3857118   

      STATUS ADDRTYPE  INTKEY  \
0    Matched    Block     nan   
1    Matched    Block     nan   
2  Unmatched    Block     nan   
3  Unmatched    Block     nan   
4  Unmatched    Block     nan   

                                                      LOCATION EXCEPTRSNCODE  \
0              BROADWAY BETWEEN E COLUMBIA ST AND BOYLSTON AVE                 
1               8TH AVE NE BETWEEN NE 45TH E ST AND NE 47TH ST                 
2                         JAMES ST BETWEEN 6TH AVE AND 7TH AVE           NaN   
3      NE NORTHGATE WAY BETWEEN 1ST AVE NE AND NE NORTHGATE DR                 
4  M L KING JR ER WAY S BETWEEN S ANGELINE ST AND S EDMUNDS ST                 

  EXCEPTRSNDESC SEVERITYCODE                    SEVERITYDESC COLLISIONTYPE  \
0           NaN            1  Property Damage Only Collision     Sideswipe   
1           NaN            1  Property Damage Only Collision    Parked Car   
2           NaN            0                         Unknown           NaN   
3           NaN            0                         Unknown           NaN   
4           NaN            0                         Unknown           NaN   

   PERSONCOUNT  PEDCOUNT  PEDCYLCOUNT  VEHCOUNT  INJURIES  SERIOUSINJURIES  \
0            2         0            0         2         0                0   
1            2         0            0         2         0                0   
2            0         0            0         0         0                0   
3            0         0            0         0         0                0   
4            0         0            0         0         0                0   

   FATALITIES                 INCDATE               INCDTTM  \
0           0  2020/01/22 00:00:00+00  1/22/2020 3:21:00 PM   
1           0  2020/01/07 00:00:00+00   1/7/2020 8:00:00 AM   
2           0  2004/01/30 00:00:00+00             1/30/2004   
3           0  2016/01/23 00:00:00+00             1/23/2016   
4           0  2020/01/26 00:00:00+00             1/26/2020   

                              JUNCTIONTYPE  SDOT_COLCODE  \
0  Mid-Block (not related to intersection)  11.000000000   
1  Mid-Block (not related to intersection)  15.000000000   
2     Mid-Block (but intersection related)  11.000000000   
3  Mid-Block (not related to intersection)  11.000000000   
4  Mid-Block (not related to intersection)  28.000000000   

                                               SDOT_COLDESC INATTENTIONIND  \
0    MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE            NaN   
1  MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE SIDESWIPE            NaN   
2    MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE            NaN   
3    MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE            NaN   
4             MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT            NaN   

  UNDERINFL  WEATHER ROADCOND                LIGHTCOND PEDROWNOTGRNT  \
0         N  Raining      Wet  Dark - Street Lights On           NaN   
1         N    Clear      Dry                 Daylight           NaN   
2       NaN      NaN      NaN                      NaN           NaN   
3       NaN      NaN      NaN                      NaN           NaN   
4       NaN      NaN      NaN                      NaN           NaN   

         SDOTCOLNUM SPEEDING ST_COLCODE  \
0               nan      NaN         11   
1               nan      NaN         32   
2 4030032.000000000      NaN        NaN   
3               nan      NaN              
4               nan      NaN              

                                                         

<h2 id="data_wrangling">Data Wrangling</h2>

Steps for working with missing data:
<ol>
    <li>Identify missing data.</li>
    <li>Deal with missing data.</li>
    <li>Correct data format.</li>
</ol>

<h3 id="identifying_missing_data">Identifying Missing Data</h3>

The missing values are converted to Python's default. We use Python's built-in functions to identify these missing values. 

In [5]:
# Test whether the collisions DataFrame has NaN values.
if df.isna().any(axis=None):
    print("DataFrame has NaN.")
else:
    print("DataFrame has no NaN.")

DataFrame has NaN.


In [6]:
# Initialize a list to store the labels for the columns with missing data.
list_of_columns_with_missing_data = list()

# For each column in the collisions DataFrame,
# if the column contains at least one NaN, 
# then add the column's label to the list.
for column in list(df.columns):
    if df[column].hasnans:
        list_of_columns_with_missing_data.append(column)

# Print the number of columns
print("Number of columns: %d" % len(df.columns))
print("List of columns labels:")
print(list(df.columns))
print()
print("Number of columns missing data: %d" % len(list_of_columns_with_missing_data))
print("List of columns missing data:")
print(list_of_columns_with_missing_data)

Number of columns: 40
List of columns labels:
['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR']

Number of columns missing data: 22
List of columns missing data:
['X', 'Y', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE', 'COLLISIONTYPE', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC']


<h3 id="deal_with_missing_data">Deal with Missing Data</h3>

<ol>
    <li>Drop the Data
        <ol>
            <li>Drop entire row.</li>
            <li>Drop entire column.</li>
        </ol>
    </li>
    <li>Replace the Data
        <ol>
            <li>Replace data by mean.</li>
            <li>Replace data by frequency.</li>
            <li>Replace data based on other functions.</li>
        </ol>
    </li>
        
</ol>

Whole columns should be dropped only if most entries in the column are empty.

In [7]:
### DELETE THIS CELL BEFORE PRODUCTION ###

# Print a list of all the column labels for the collisions DataFrame.
print(list(df.columns))

['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR']


In [8]:
# Drop any column from the collisions DataFrame if it satisfies at least one of the following conditions:
# 1) more than 15% of the column's data is NaN;
# 2) the column only contains unique identification keys;
# 3) the column's data is naturally categorical but does not fit into a small (< 50) number of categories;
# 4) infromation in one column is redundant because it is already represented by another column;
# 5) it is not clear how to interpret the column's data.

list_of_columns_to_drop = [\
                           "STATUS",\
                           "OBJECTID",\
                           "INCKEY",\
                           "COLDETKEY",\
                           "REPORTNO",\
                           "INTKEY",\
                           "LOCATION",\
                           "EXCEPTRSNCODE",\
                           "EXCEPTRSNDESC",\
                           "SEVERITYDESC",\
                           "INCDATE",\
                           "INATTENTIONIND",\
                           "UNDERINFL",\
                           "PEDROWNOTGRNT",\
                           "SDOTCOLNUM",\
                           "SPEEDING",\
                           "SEGLANEKEY",\
                           "CROSSWALKKEY"]

In [9]:
#NOTE: drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
# Drop the selected columns from the collisions DataFrame
# and store the result in a new DataFrame.
df_after_drop_columns = df.drop(columns=list_of_columns_to_drop, inplace=False)

In [10]:
# NOTE: dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)

# Drop any row that contains at least one NaN.
#print("Number of columns: %d" % len(list(df_after_drop_columns.columns)))
df_after_drop_columns_and_rows = df_after_drop_columns.dropna(axis="index", how="any", thresh=None, subset=None, inplace=False)

# Drop any row with SEVERITYCODE equal to 0.
#df_after_drop_columns[df_after_drop_columns["SEVERITYCODE"].astype(np.dtype("int64))]
#df_after_drop_columns.info()

<h3 id="correct_data_format">Correct Data Format</h3>

Ensure that each data type is appropriate for the corresponding feature.
Convert integer data to "ordered" categorical types, e.g. SEVERITYCODE,
especially if the "integer ordering" of the original data is inappropriate.

If data represents date, time, or date/time information, then convert the data to the appropriate datetime representation.

In [11]:
# Create new DataFrame to store converted data types.
df_converted = pd.DataFrame()

for column in list(df_after_drop_columns_and_rows.columns):
     # Cast column "ST_COLCODE" to type category.
    if column in ["SDOT_COLCODE"]:
        df_converted["SDOT_COLCODE"] = df_after_drop_columns_and_rows["SDOT_COLCODE"].astype('category')
    # Cast columns "INCDATE" and "INCDTTM" to type datetime.
    elif column in ["INCDATE", "INCDTTM"]:
        df_converted[column] = pd.to_datetime(df_after_drop_columns_and_rows[column], infer_datetime_format=True)
    # Cast columns of type object to type category.
    elif (df_after_drop_columns_and_rows[column].dtype in [np.dtype('object')]):
        df_converted[column] = df_after_drop_columns_and_rows[column].astype('category')
    # Copy all other columns to new DataFrame without changing their types.
    else:
        df_converted[column] = df_after_drop_columns_and_rows[column]

In [12]:
# Display info about new DataFrame after casting objects to category or date
df_converted.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185318 entries, 0 to 221388
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   X                185318 non-null  float64       
 1   Y                185318 non-null  float64       
 2   ADDRTYPE         185318 non-null  category      
 3   SEVERITYCODE     185318 non-null  category      
 4   SEVERITYDESC     185318 non-null  category      
 5   COLLISIONTYPE    185318 non-null  category      
 6   PERSONCOUNT      185318 non-null  int64         
 7   PEDCOUNT         185318 non-null  int64         
 8   PEDCYLCOUNT      185318 non-null  int64         
 9   VEHCOUNT         185318 non-null  int64         
 10  INJURIES         185318 non-null  int64         
 11  SERIOUSINJURIES  185318 non-null  int64         
 12  FATALITIES       185318 non-null  int64         
 13  INCDATE          185318 non-null  datetime64[ns]
 14  INCDTTM          185

In [13]:
# Create DataFrame of categorical columns.
df_categorical = df_converted.select_dtypes(include="category")

In [14]:
df_categorical.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185318 entries, 0 to 221388
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   ADDRTYPE       185318 non-null  category
 1   SEVERITYCODE   185318 non-null  category
 2   SEVERITYDESC   185318 non-null  category
 3   COLLISIONTYPE  185318 non-null  category
 4   JUNCTIONTYPE   185318 non-null  category
 5   SDOT_COLCODE   185318 non-null  category
 6   SDOT_COLDESC   185318 non-null  category
 7   WEATHER        185318 non-null  category
 8   ROADCOND       185318 non-null  category
 9   LIGHTCOND      185318 non-null  category
 10  ST_COLCODE     185318 non-null  category
 11  ST_COLDESC     185318 non-null  category
 12  HITPARKEDCAR   185318 non-null  category
dtypes: category(13)
memory usage: 3.7 MB


In [15]:
# If SEVERITYCODE == 0, then remove the row from collisions DataFrame.

# Remove column SEVERITYDESC because of redundancy with column SEVERITYCODE.



In [16]:
# For each categorical column in DataFrame df_categorical, print value_counts.
for column in list(df_categorical.columns):
    print(df_categorical[column].value_counts(normalize=True, dropna=False))
    print(df_categorical[column].value_counts(normalize=False, dropna=False))
    print()

Block          0.647157858
Intersection   0.352842142
Name: ADDRTYPE, Length: 2, dtype: float64
Block           119930
Intersection     65388
Name: ADDRTYPE, Length: 2, dtype: int64

1    0.677327621
2    0.304800397
2b   0.016080467
3    0.001780723
0    0.000010792
Name: SEVERITYCODE, Length: 5, dtype: float64
1     125521
2      56485
2b      2980
3        330
0          2
Name: SEVERITYCODE, Length: 5, dtype: int64

Property Damage Only Collision   0.677327621
Injury Collision                 0.304800397
Serious Injury Collision         0.016080467
Fatality Collision               0.001780723
Unknown                          0.000010792
Name: SEVERITYDESC, Length: 5, dtype: float64
Property Damage Only Collision    125521
Injury Collision                   56485
Serious Injury Collision            2980
Fatality Collision                   330
Unknown                                2
Name: SEVERITYDESC, Length: 5, dtype: int64

Parked Car   0.234272979
Angles       0.190111052
Rear 

#### Feature before One Hot Encoding